In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [16]:
import gradio as gr
import networkx as nx
from itertools import combinations
import matplotlib.pyplot as plt
import tempfile

def create_graph(exams, relations):
    G = nx.Graph()
    G.add_nodes_from(exams)
    G.add_edges_from(relations)
    return G

# Fonction pour planifier les examens
def schedule_exams(graph):
    coloring = nx.coloring.greedy_color(graph, strategy="largest_first")
    schedule = {}
    for exam, slot in coloring.items():
        if slot not in schedule:
            schedule[slot] = []
        schedule[slot].append(exam)
    return schedule, coloring

# Fonction pour visualiser le graphe
def visualize_graph(graph, coloring):
    pos = nx.spring_layout(graph)
    plt.figure(figsize=(6, 5))
    nx.draw(
        graph,
        pos,
        with_labels=True,
        node_color=[coloring[node] for node in graph.nodes()],
        cmap=plt.cm.rainbow,
        node_size=600,
        font_size=8
    )
    plt.title("Relations entre examens")
    return plt.gcf()

# Fonction pour enregistrer le graphe sous forme de fichier .png
def save_graph_as_png(graph, coloring):
    fig = visualize_graph(graph, coloring)
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
    fig.savefig(temp_file.name)
    plt.close(fig)
    return temp_file.name

# Fonction pour générer les relations dynamiques entre les examens
def generate_relations(num_exams):
    exams = [f"Examen {i}" for i in range(1, num_exams + 1)]
    relations = [f"{i} - {j}" for i, j in combinations(exams, 2)]
    return relations

# Fonction pour générer la planification et la visualisation
def generate_schedule(num_exams, relations_input):
    exams = [f"Examen {i}" for i in range(1, num_exams + 1)]
    relations = []
    for i, j in combinations(exams, 2):
        if f"{i} - {j}" in relations_input or f"{j} - {i}" in relations_input:
            relations.append((i, j))

    graph = create_graph(exams, relations)
    schedule, coloring = schedule_exams(graph)

    fig = visualize_graph(graph, coloring)

    schedule_text = ""
    for slot, exams in schedule.items():
        schedule_text += f"Créneau {slot + 1} : {', '.join(exams)}\n"

    return schedule_text, fig, graph, coloring

# Interface
def create_interface():
    with gr.Blocks() as demo:
        gr.Markdown("## Planificateur d'Horaires d'Examens")

        gr.Markdown(
            "1. Entrez le nombre d'examens, puis cliquez sur **Générer les relations**.\n"
            "2. Cochez les relations entre les examens et cliquez sur **Générer la planification**.\n"
            "3. Cliquez sur **Enregistrer le graphe** pour télécharger l'image."
        )

        num_exams_input = gr.Number(label="Nombre d'examens", value=5, minimum=1, maximum=30)

        generate_relations_button = gr.Button("Générer les relations", size="sm")
        relations_input = gr.CheckboxGroup(label="Relations entre les examens", interactive=True)
        generate_schedule_button = gr.Button("Générer la planification", size="sm")
        schedule_output = gr.Textbox(label="Planification des Examens", lines=5, interactive=False)
        graph_output = gr.Plot(label="Visualisation du Graphe")
        save_graph_button = gr.Button("Enregistrer le graphe", size="sm")

        download_output = gr.File(label="Télécharger le graphe")

        graph_state = gr.State()
        coloring_state = gr.State()

        # génération des relations
        def on_generate_relations(num_exams):
            relations = generate_relations(num_exams)
            return gr.update(choices=relations)

        generate_relations_button.click(
            fn=on_generate_relations,
            inputs=num_exams_input,
            outputs=relations_input
        )

        # génération de la planification
        def on_generate_schedule(num_exams, relations_input):
            schedule_text, fig, graph, coloring = generate_schedule(num_exams, relations_input)
            return schedule_text, fig, graph, coloring

        generate_schedule_button.click(
            fn=on_generate_schedule,
            inputs=[num_exams_input, relations_input],
            outputs=[schedule_output, graph_output, graph_state, coloring_state]
        )

        # enregistrer le graphe
        def on_save_graph(graph, coloring):
            file_path = save_graph_as_png(graph, coloring)
            return file_path

        save_graph_button.click(
            fn=on_save_graph,
            inputs=[graph_state, coloring_state],
            outputs=download_output
        )

    demo.launch()

create_interface()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9eb4917dfbd8eda1a6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
